In [1]:
import pandas as pd
import json
import os

In [2]:
# Define the file names
file_names = ["Cupertino.csv", "District Office.csv", "Fremont.csv", "Homestead.csv", "Lynbrook.csv", "Monta Vista.csv"]

# Directory paths
input_directory = '/mnt/data/'
output_directory = '/mnt/data/processed/'

# Create output directory if not exists
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to process CSV files
def process_csv(file_name):
    try:
        # Read the CSV file
        df = pd.read_csv(input_directory + file_name)
        
        # Print columns for debugging
        print(f"Processing {file_name}")
        print(f"Columns: {df.columns.tolist()}")
        
        # Extract Date and CO2 columns
        if file_name == "District Office.csv":
            co2_column = 'Total'
        else:
            co2_column = [col for col in df.columns if 'CO2' in col or 'Carbon' in col]
            if not co2_column:
                print(f"No CO2/Carbon column found in {file_name}")
                return
            co2_column = co2_column[0]
        
        df_co2 = df[[' Date', co2_column]].copy()
        
        # Rename columns
        df_co2.columns = ['Date', 'CO2']
        
        # Save to new CSV
        output_csv_file = output_directory + file_name.replace('.csv', '_co2.csv')
        df_co2.to_csv(output_csv_file, index=False)
        
        # Convert to JSON
        json_data = df_co2.to_json(orient='records')
        output_json_file = output_csv_file.replace('.csv', '.json')
        with open(output_json_file, 'w') as json_file:
            json_file.write(json_data)
        
        print(f"Processed {file_name} successfully.")
    
    except Exception as e:
        print(f"Error processing {file_name}: {e}")

# Process each file
for file_name in file_names:
    process_csv(file_name)

Error processing Cupertino.csv: [Errno 2] No such file or directory: '/mnt/data/Cupertino.csv'
Error processing District Office.csv: [Errno 2] No such file or directory: '/mnt/data/District Office.csv'
Error processing Fremont.csv: [Errno 2] No such file or directory: '/mnt/data/Fremont.csv'
Error processing Homestead.csv: [Errno 2] No such file or directory: '/mnt/data/Homestead.csv'
Error processing Lynbrook.csv: [Errno 2] No such file or directory: '/mnt/data/Lynbrook.csv'
Error processing Monta Vista.csv: [Errno 2] No such file or directory: '/mnt/data/Monta Vista.csv'


In [31]:
import pandas as pd
import json

ev_data = pd.read_csv('./fuhsd_22_23_ev_data.csv')
print(ev_data.shape[0])
#print(ev_data.columns)
# print(ev_data.head())
#print(ev_data[['GHGs avoided (lbs)']].sum()) # sum of GHGs avoided over the year

# convert data types
ev_data['AVG KWh Delivered Per Session (kWh)'] = ev_data['AVG KWh Delivered Per Session (kWh)'].astype(float)
ev_data['AVG Session Duration (minutes)'] = ev_data['AVG Session Duration (minutes)'].astype(float)

# drop nonzeros -- they unnecessarily drag down average, as there's no charging happening
kwh_nonzero_avg = ev_data[ev_data['AVG KWh Delivered Per Session (kWh)'] > 0.01]['AVG KWh Delivered Per Session (kWh)'].mean()
dur_nonzero_avg = ev_data[ev_data['AVG Session Duration (minutes)'] > 0.01]['AVG Session Duration (minutes)'].mean()

print(kwh_nonzero_avg, dur_nonzero_avg)

# convert duration to hours
dur_hours = dur_nonzero_avg/60
dur_hours

# total GHGs avoided PER CHARGER PER DAY = user_entered_hrs * kwh/hr (<-- total kWh per charger per day, on average) * 0.000699 * 2204.62
kwh_per_hr = kwh_nonzero_avg/dur_hours
print("KWH PER HOUR", kwh_per_hr) # ~2.695
ghgs_per_charger_per_day = kwh_per_hr * 2204.62 * 0.000699

ghgs_per_charger_per_day # ~4.1533 is the DEFAULT VALUE for the slider

# so the slider formula will be:
# user_entered_hrs * 2.695 * 1.79 *  2204.62 * 0.000404 * user_entered_charger_num

# for trees it'll be user_entered_hrs * 2.695 * 0.0116

'''
ok so now
'''


292
13.016829293800502 289.7839189371955
KWH PER HOUR 2.695145267178533


np.float64(4.1532980400900685)

In [17]:
total = 0
schools=['Cupertino', 'Fremont', 'Lynbrook', 'District Office']
for s in schools:
    df = pd.read_csv('./data_processed/' + s + '_co2.csv')
    df[['CO2']] = df[['CO2']].astype(float)
    print(df.dtypes)
    sum_co2 = df[['CO2']].sum()
    print(sum_co2)
    total += sum_co2

print("TOTAL IS", total)

Date     object
CO2     float64
dtype: object
CO2    2224278.99
dtype: float64
Date     object
CO2     float64
dtype: object
CO2    3651025.68
dtype: float64
Date     object
CO2     float64
dtype: object
CO2    2212113.15
dtype: float64
Date     object
CO2     float64
dtype: object
CO2    650868.99
dtype: float64
TOTAL IS CO2    8738286.81
dtype: float64


In [6]:
# Emissions data for each school
emissions_data = {
    "Fremont High School": 266406.13,
    "Cupertino High School": 1138238.97,
    "Monta Vista High School": 3281710.55,
    "Lynbrook High School": 999497.58,
    "Homestead High School": 1243537.7
}

# Save individual emissions data to JSON files
os.makedirs("data_processed", exist_ok=True)
for school, emissions in emissions_data.items():
    file_name = f"data_processed/{school.replace(' ', '_').lower()}_total_emissions.json"
    with open(file_name, 'w') as f:
        json.dump({"total_emissions": emissions}, f)

# Calculate and save total emissions for the district
total_emissions = sum(emissions_data.values())
with open("data_processed/FUHSD_total_emissions.json", 'w') as f:
    json.dump({"total_emissions": total_emissions}, f)

# Load and analyze district office emissions data for 2023
with open('data_processed/District Office_co2.json') as f:
    district_office_data = json.load(f)

district_office_2023_emissions = sum(
    entry["CO2"] for entry in district_office_data if entry["Date"].endswith("2023")
)

# Save district office 2023 emissions to JSON
with open("data_processed/district_office_emissions.json", 'w') as f:
    json.dump({"total_emissions_2023": district_office_2023_emissions}, f)

# Add district office 2023 emissions to the total emissions
total_emissions += district_office_2023_emissions

# Save updated total emissions for the district
with open("data_processed/FUHSD_total_emissions.json", 'w') as f:
    json.dump({"total_emissions": total_emissions}, f)

# Save district office 2023 emissions to JSON
with open("data_processed/district_office_emissions.json", 'w') as f:
    json.dump({"total_emissions_2023": district_office_2023_emissions}, f)

print("Emissions data has been processed and saved.")

Emissions data has been processed and saved.


In [14]:
# found through the US gov: https://www.epa.gov/energy/frequent-questions-epas-greenhouse-gas-equivalencies-calculator#:~:text=According%20to%20the%20Calculations%20and,per%20year%20from%20electricity%20alone., and https://www.fs.usda.gov/about-agency/features/trees-are-climate-change-carbon-storage-heroes#:~:text=In%20one%20year%2C%20a%20mature,atmosphere%2C%20like%20fire%20or%20decomposition.
CO2_PER_TREE = 21.77209  # CO2 offset per tree in kg per year
CO2_PER_HOME_YEARLY = 10969.99097832  # CO2 emissions per home in kg per year 

# Load the total emissions for the district
with open("data_processed/FUHSD_total_emissions.json") as f:
    fuhsd_total_emissions = json.load(f)["total_emissions"]

# Calculate the number of trees equivalent
trees_equivalent = fuhsd_total_emissions / CO2_PER_TREE

# Calculate the number of homes equivalent
homes_equivalent = fuhsd_total_emissions / CO2_PER_HOME_YEARLY

# Save the results to JSON files
os.makedirs("data_processed", exist_ok=True)

with open("data_processed/FUHSD_trees_equivalent.json", 'w') as f:
    json.dump({"trees_equivalent": trees_equivalent}, f)

with open("data_processed/FUHSD_homes_equivalent.json", 'w') as f:
    json.dump({"homes_equivalent": homes_equivalent}, f)

print("Equivalent data has been calculated and saved.")

Equivalent data has been calculated and saved.


In [15]:
#optimal temps from silvera, HVAC percent from study silvera sent, degree impact from this study: https://research.uga.edu/news/turn-up-the-thermostat-lower-energy-costs-no-complaints/#:~:text=%E2%80%9COver%20the%20years%2C%20studies%20have,settings)%2C%E2%80%9D%20he%20said.

OPTIMAL_TEMP_SUMMER = 72
OPTIMAL_TEMP_WINTER = 68
HVAC_BASE_PERCENTAGE = 0.33
HVAC_DEGREE_IMPACT = 0.03

# Load the total emissions for the district
with open("data_processed/FUHSD_total_emissions.json") as f:
    fuhsd_total_emissions = json.load(f)["total_emissions"]

# Calculate base HVAC emissions (33% of total district emissions)
base_hvac_emissions = fuhsd_total_emissions * HVAC_BASE_PERCENTAGE

# Calculate emissions impact per degree deviation from optimal
hvac_impact_per_degree = fuhsd_total_emissions * HVAC_DEGREE_IMPACT

# Calculate emissions for different temperatures
temperatures = range(OPTIMAL_TEMP_WINTER, OPTIMAL_TEMP_SUMMER + 1)
hvac_emissions_per_temp = {
    temp: base_hvac_emissions + (hvac_impact_per_degree * (temp - OPTIMAL_TEMP_WINTER))
    for temp in temperatures
}

# Print the calculated HVAC emissions per temperature
hvac_emissions_per_temp

{68: 2334600.4143000003,
 69: 2546836.8156000003,
 70: 2759073.2169000003,
 71: 2971309.6182000004,
 72: 3183546.0195000004}

In [10]:
# Constants for house emissions
CO2_PER_HOME_YEARLY = 10969.99097832  # CO2 emissions per home in kg per year

# Calculate emissions per degree deviation from optimal
hvac_impact_per_degree = fuhsd_total_emissions * HVAC_DEGREE_IMPACT

# Calculate HVAC emissions and corresponding homes powered for different temperatures
hvac_emissions_per_temp = {
    temp: {
        "emissions": base_hvac_emissions + (hvac_impact_per_degree * (temp - OPTIMAL_TEMP_WINTER)),
        "homes_powered": (base_hvac_emissions + (hvac_impact_per_degree * (temp - OPTIMAL_TEMP_WINTER))) / CO2_PER_HOME_YEARLY
    }
    for temp in range(OPTIMAL_TEMP_WINTER, OPTIMAL_TEMP_SUMMER + 1)
}

# Print the calculated HVAC emissions and homes powered per temperature
hvac_emissions_per_temp


{68: {'emissions': 2334600.4143000003, 'homes_powered': 96.53208222467833},
 69: {'emissions': 2546836.8156000003, 'homes_powered': 105.30772606328546},
 70: {'emissions': 2759073.2169000003, 'homes_powered': 114.08336990189258},
 71: {'emissions': 2971309.6182000004, 'homes_powered': 122.8590137404997},
 72: {'emissions': 3183546.0195000004, 'homes_powered': 131.63465757910683}}

In [11]:
CO2_PER_TREE_YEARLY = 21.77209  # CO2 offset per tree in kg per year

# Calculate HVAC emissions and corresponding homes/trees powered for different temperatures
hvac_emissions_per_temp = {
    temp: {
        "emissions": round(base_hvac_emissions + (hvac_impact_per_degree * (temp - OPTIMAL_TEMP_WINTER))),
        "homes_powered": round((base_hvac_emissions + (hvac_impact_per_degree * (temp - OPTIMAL_TEMP_WINTER))) / CO2_PER_HOME_YEARLY),
        "trees_required": round((base_hvac_emissions + (hvac_impact_per_degree * (temp - OPTIMAL_TEMP_WINTER))) / CO2_PER_TREE_YEARLY)
    }
    for temp in range(OPTIMAL_TEMP_WINTER, OPTIMAL_TEMP_SUMMER + 1)
}

# Print the calculated HVAC emissions, homes powered, and trees required per temperature
hvac_emissions_per_temp

{68: {'emissions': 2334600, 'homes_powered': 97, 'trees_required': 48638},
 69: {'emissions': 2546837, 'homes_powered': 105, 'trees_required': 53059},
 70: {'emissions': 2759073, 'homes_powered': 114, 'trees_required': 57481},
 71: {'emissions': 2971310, 'homes_powered': 123, 'trees_required': 61902},
 72: {'emissions': 3183546, 'homes_powered': 132, 'trees_required': 66324}}